# Functions 

## Get applicable units per quantity

In [92]:
def GetApplicableUnits(quantitiyURI):
    from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON
    sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
    sparql.setMethod(POST)
    sparql.setQuery(f"""
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?applicableUnit
        WHERE {{
          <{quantitiyURI}> qudt:applicableUnit ?applicableUnit.
        }} 
        LIMIT 999
    """)
    sparql.setReturnFormat(JSON)
    applicableUnitsRaw = sparql.query().convert()
    #return applicableUnitsRaw
    ApplicableUnitsDictionary = {}
    for unit in applicableUnitsRaw["results"]["bindings"]:
        ApplicableUnitsDictionary[unit["applicableUnit"]["value"]] = {}
    
    return ApplicableUnitsDictionary


In [2]:
def GetRunaways(inDict, notInDict, printCount, printKeys, printValues):
    runawayDict = {}
    for key,value in inDict.items():
        if key not in notInDict.keys():
            runawayDict[key] = value

    if printCount==True:
        print(len(runawayDict.keys()))
    
    if printKeys==True:
        for idx in runawayDict.keys():
            print(idx)
        
    
    if printValues==True:
        for idx in runawayDict.values():
            print(idx)
        
    return runawayDict



In [3]:
def MergeRunaways(Dict1, Dict2, printCount, printKeys, printValues):
    runawayDict = Dict1
    for key,value in Dict2.items():
        if key not in runawayDict.keys():
            runawayDict[key] = value

    if printCount==True:
        print(len(runawayDict.keys()))
    
    if printKeys==True:
        for idx in runawayDict.keys():
            print(idx)
        
    
    if printValues==True:
        for idx in runawayDict.values():
            print(idx)
        
    return runawayDict

# Variables

## read from dictionaries

In [4]:
# set value to compare results
setKey = 'quantityKind'
setValue = 'hasDimensionVector'

# function runaways print settings
printCount= True
printKeys= False
printValues= False

# Queries

## Query 1 quantities + dimension vectors, key: quantities

In [5]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?quantityKind ?hasDimensionVector
WHERE {
	?quantityKind  rdf:type qudt:QuantityKind ;
		qudt:hasDimensionVector ?hasDimensionVector ;
} 
ORDER BY ASC (?quantityKind)
LIMIT 999
""")
sparql.setReturnFormat(JSON)
quantityKindDim = sparql.query().convert()


In [6]:
quantityKindDimDict = {}
for idx in quantityKindDim['results']['bindings']:
    if idx[setKey]['value'] not in quantityKindDimDict.keys():
        quantityKindDimDict[idx[setKey]['value']] = {idx[setValue]['value']: {'Query 1 quantityKind'}}

print(len(quantityKindDimDict.keys()))
#print(quantityKindDimDict.values())

686


## Query 2 Units + additional information including quantity kind and dimension vector, key: units

In [7]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX qudt: <http://qudt.org/schema/qudt/>

SELECT ?quantityKind ?quantityName ?isBaseUnit ?hasDimensionVector ?unitIRI ?unitName ?conversionMultiplier 
WHERE {
  	?unitIRI  rdf:type qudt:Unit ;
    	qudt:conversionMultiplier	?conversionMultiplier ;
		qudt:hasQuantityKind 		?quantityKind ;
         qudt:hasDimensionVector	?hasDimensionVector ;
  	FILTER ((?conversionMultiplier)!=0.0 )
    BIND ( IF(?conversionMultiplier = 1.0, True, False)  AS ?isBaseUnit)
  	BIND ( REPLACE(STR(?unitIRI), "^.*unit/([^/]+).*$", "$1") AS ?unitName)
    BIND ( REPLACE(STR(?quantityKind), "^.*quantitykind/([^/]+).*$", "$1") AS ?quantityName)
} 
ORDER BY ASC (?quantityKind)
LIMIT 10000
""")
sparql.setReturnFormat(JSON)
unit = sparql.query().convert()


In [8]:
unitDimDict = {}
for idx in unit['results']['bindings']:
    if idx[setKey]['value'] not in unitDimDict.keys():
        unitDimDict[idx[setKey]['value']] = {idx[setValue]['value']: {'Query 2 units'}}

print(len(unitDimDict.keys()))


402


## Query 3 dimension vectors incl. reference quantityKind, key: dimensions

In [9]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT  ?hasDimensionVector ?quantityKind
WHERE {
    ?hasDimensionVector  rdf:type qudt:QuantityKindDimensionVector_SI ;
    qudt:hasReferenceQuantityKind ?quantityKind ;
} 
ORDER BY ASC (?quantityKind)
LIMIT 400000
""")
sparql.setReturnFormat(JSON)
dimension = sparql.query().convert()


In [10]:
dimensionDimDict = {}

for idx in dimension['results']['bindings']:
    if idx[setKey]['value'] not in dimensionDimDict.keys():
        dimensionDimDict[idx[setKey]['value']] = {idx[setValue]['value']: {'Query 3 dimensions'}}

    
print(len(dimensionDimDict.keys()))
#print(dimensionDimDict.values())

130


## Query 4 all quantities 

In [11]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?quantityKind 
WHERE {
	?quantityKind  rdf:type qudt:QuantityKind ;
} 
ORDER BY ASC (?quantityKind)
LIMIT 999
""")
sparql.setReturnFormat(JSON)
quantityKind = sparql.query().convert()

In [12]:
quantityKindDict = {}
for idx in quantityKind['results']['bindings']:
    if idx[setKey]['value'] not in quantityKindDict.keys():
        quantityKindDict[idx[setKey]['value']] = {}

print(len(quantityKindDict.keys()))
#print(quantityKindDimDict.values())

861


# Find runaways

## determine query 1: runaways from quantityKind

In [13]:
print('\nin units but not in quantityKind')      
dimInUnitsNotInQuantityKind = GetRunaways(unitDimDict, quantityKindDimDict, printCount, True, printValues)

print('\nin dimensions but not in quantityKind')    
dimInDimensionsNotInQuantityKind = GetRunaways(dimensionDimDict, quantityKindDimDict, printCount, True, True)

print('\nmerged unique runaways in quantityKind')
runawaysQuantityKind = MergeRunaways(dimInUnitsNotInQuantityKind, dimInDimensionsNotInQuantityKind, printCount, printKeys, printValues)


in units but not in quantityKind
23
http://qudt.org/vocab/quantitykind/InverseTemperature
http://qudt.org/vocab/quantitykind/LinearExpansionCoefficient
http://qudt.org/vocab/quantitykind/MassAbsorptionCoefficient
http://qudt.org/vocab/quantitykind/MassAmountOfSubstance
http://qudt.org/vocab/quantitykind/MassEnergyTransferCoefficient
http://qudt.org/vocab/quantitykind/Mobility
http://qudt.org/vocab/quantitykind/MolarAbsorptionCoefficient
http://qudt.org/vocab/quantitykind/MolarAttenuationCoefficient
http://qudt.org/vocab/quantitykind/Permeability
http://qudt.org/vocab/quantitykind/PowerAreaPerSolidAngle
http://qudt.org/vocab/quantitykind/PressureCoefficient
http://qudt.org/vocab/quantitykind/Radiance
http://qudt.org/vocab/quantitykind/RelativePressureCoefficient
http://qudt.org/vocab/quantitykind/ResidualResistivity
http://qudt.org/vocab/quantitykind/Resistivity
http://qudt.org/vocab/quantitykind/SoundExposure
http://qudt.org/vocab/quantitykind/SpecificAcousticImpedance
http://qudt.org

## determine query 2: runaways from units

In [14]:
print('\nin quantityKind but not in units')       
dimInQuantityKindNotInUnits = GetRunaways(quantityKindDimDict, unitDimDict, printCount, printKeys, printValues)

print('\nin dimensions but not in units')  
dimInDimensionsNotInUnits = GetRunaways(dimensionDimDict, unitDimDict, printCount, printKeys, printValues)

print('\nmerged unique runaways in units')
runawaysUnits = MergeRunaways(dimInQuantityKindNotInUnits, dimInDimensionsNotInUnits, printCount, printKeys, printValues)


in quantityKind but not in units
307

in dimensions but not in units
9

merged unique runaways in units
308


## determine query 3: runaways from dimensions

In [15]:
print('\nin quantityKind but not in dimensions')       
dimInQuantityKindNotInDimensions = GetRunaways(quantityKindDimDict, dimensionDimDict, printCount, printKeys, printValues)

print('\nin units but not in dimensions')  
dimInUnitsNotInDimensions = GetRunaways(unitDimDict, dimensionDimDict, printCount, printKeys, printValues)

print('\nmerged unique runaways in dimensions')
runawaysDimensions = MergeRunaways(dimInQuantityKindNotInDimensions, dimInUnitsNotInDimensions, printCount, printKeys, printValues)


in quantityKind but not in dimensions
557

in units but not in dimensions
281

merged unique runaways in dimensions
580


### Find all quantityKind elements whitch have no dimension

In [42]:
print('\nin quantityKindDict but not in quantityKindDimDict')      
quantityKindWithoutDim= GetRunaways(quantityKindDict, quantityKindDimDict, printCount, True, printValues)


in quantityKindDict but not in quantityKindDimDict
175
http://qudt.org/vocab/quantitykind/ActivityThresholds
http://qudt.org/vocab/quantitykind/Adaptation
http://qudt.org/vocab/quantitykind/AngleOfAttack
http://qudt.org/vocab/quantitykind/AngleOfOpticalRotation
http://qudt.org/vocab/quantitykind/AngularDistance
http://qudt.org/vocab/quantitykind/ApogeeRadius
http://qudt.org/vocab/quantitykind/Asset
http://qudt.org/vocab/quantitykind/AuditoryThresholds
http://qudt.org/vocab/quantitykind/AverageSpecificImpulse
http://qudt.org/vocab/quantitykind/BevelGearPitchAngle
http://qudt.org/vocab/quantitykind/BraggAngle
http://qudt.org/vocab/quantitykind/BucklingFactor
http://qudt.org/vocab/quantitykind/CENTER-OF-GRAVITY_X
http://qudt.org/vocab/quantitykind/CENTER-OF-GRAVITY_Y
http://qudt.org/vocab/quantitykind/CENTER-OF-GRAVITY_Z
http://qudt.org/vocab/quantitykind/Capacity
http://qudt.org/vocab/quantitykind/ClosestApproachRadius
http://qudt.org/vocab/quantitykind/CompressibilityFactor
http://qudt

In [17]:
print('\nin units but not in quantityKind')      
dimInUnitsNotInQuantityKind = GetRunaways(unitDimDict, quantityKindDict, printCount, True, printValues)

print('\nin dimensions but not in quantityKind')    
dimInDimensionsNotInQuantityKind = GetRunaways(dimensionDimDict, quantityKindDict, printCount, True, True)

print('\nmerged unique runaways in quantityKind')
runawaysQuantityKind = MergeRunaways(dimInUnitsNotInQuantityKind, dimInDimensionsNotInQuantityKind, printCount, printKeys, printValues)


in units but not in quantityKind
0

in dimensions but not in quantityKind
1
http://qudt.org/vocab/quantitykind/VoltageLoss
{'http://qudt.org/vocab/dimensionvector/A0E2L0I0M0H0T1D0': {'Query 3 dimensions'}}

merged unique runaways in quantityKind
1
